In [1]:
from copy import deepcopy
import subprocess
import numpy as np
import open3d as o3d
import trimesh
import os


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [9]:
print(o3d.__version__)

0.18.0


In [2]:
def subdivide_surface_fitting(decimated_mesh, target_mesh, iterations=1):
    subdivided_mesh = o3d.geometry.TriangleMesh()
    subdivided_mesh = o3d.geometry.TriangleMesh.subdivide_midpoint(decimated_mesh, number_of_iterations=iterations)
    print(subdivided_mesh)
    subdivided_mesh.compute_vertex_normals()
    #o3d.visualization.draw_geometries([subdivided_mesh])
    
    pcd_target = o3d.geometry.PointCloud()
    pcd_target.points = o3d.utility.Vector3dVector(target_mesh.vertices)
    pcd_tree = o3d.geometry.KDTreeFlann(pcd_target)
    subdivided_vertices = np.array(subdivided_mesh.vertices)
    target_vertices = np.array(target_mesh.vertices)
    fitting_vertices = deepcopy(subdivided_vertices)
    
    for i in range(0, len(subdivided_vertices)):
        [k, index, _] = pcd_tree.search_knn_vector_3d(subdivided_vertices[i], 1)
        fitting_vertices[i] = target_vertices[np.asarray(index)]
        #print(fitting_vertices[i], target_vertices[index])
    subdivided_mesh.vertices = o3d.utility.Vector3dVector(fitting_vertices)
    return subdivided_mesh

In [5]:
load_reference_mesh = o3d.io.read_triangle_mesh(r'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer/reference_mesh.obj', enable_post_processing = False)
print(load_reference_mesh)
o3d.io.write_triangle_mesh(r'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer/reference_mesh.obj', load_reference_mesh, write_vertex_normals=False, write_vertex_colors=False, write_triangle_uvs=False)

TriangleMesh with 20002 points and 40000 triangles.


True

In [7]:
load_reference_mesh = o3d.io.read_triangle_mesh(r'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer/reference_mesh.obj', enable_post_processing = False)
print(np.array(load_reference_mesh.vertices))

[[ 0.0791106   0.90974599 -0.477777  ]
 [ 0.0861452   0.90974599 -0.48245999]
 [ 0.0861452   0.905756   -0.48109001]
 ...
 [-0.127333    1.02546     0.26905501]
 [ 0.174541   -0.25041699  0.27527499]
 [-0.0575035   0.39899501  0.31645399]]


In [3]:
def read_triangle_mesh_with_trimesh(avatar_name,enable_post_processing=False):
    # EDIT: next 4 lines replace to maintain order even in case of degenerate and non referenced
    # scene_patch = trimesh.load(avatar_name,process=enable_post_processing)
    if enable_post_processing:
        scene_patch = trimesh.load(avatar_name,process=True)
    else:
        scene_patch = trimesh.load(avatar_name,process=False,maintain_order=True) 
    mesh = o3d.geometry.TriangleMesh(
        o3d.utility.Vector3dVector(scene_patch.vertices),
        o3d.utility.Vector3iVector(scene_patch.faces)
    ) 
    if scene_patch.vertex_normals.size:
        mesh.vertex_normals = o3d.utility.Vector3dVector(scene_patch.vertex_normals.copy())
    if scene_patch.visual.defined:
        # either texture or vertex colors if no uvs present.
        if scene_patch.visual.kind == 'vertex':
            mesh.vertex_colors = o3d.utility.Vector3dVector(scene_patch.visual.vertex_colors[:,:3]/255) # no alpha channel support
        elif scene_patch.visual.kind == 'texture':
            uv = scene_patch.visual.uv
            if uv.shape[0] == scene_patch.vertices.shape[0]:
                mesh.triangle_uvs = o3d.utility.Vector2dVector(uv[scene_patch.faces.flatten()])
            elif uv.shape[0] != scene_patch.faces.shape[0] * 3:
                assert False
            else:
                mesh.triangle_uvs = o3d.utility.Vector2dVector(uv)
                if scene_patch.visual.material is not None and scene_patch.visual.material.image is not None:
                    if scene_patch.visual.material.image.mode == 'RGB':
                        mesh.textures = [o3d.geometry.Image(np.asarray(scene_patch.visual.material.image))]
                    else:
                        assert False
        else:
            assert False
    return mesh

In [90]:
load_reference_mesh_trimesh = read_triangle_mesh_with_trimesh(r'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer/reference_mesh.obj', enable_post_processing = False)

print(np.array(load_reference_mesh_trimesh.vertices).__len__())
decimated_reference_mesh = o3d.geometry.TriangleMesh.simplify_quadric_decimation(load_reference_mesh_trimesh, 10000, boundary_weight= 8000)
print(decimated_reference_mesh)

subdivided_mesh = o3d.geometry.TriangleMesh.subdivide_midpoint(decimated_reference_mesh, number_of_iterations=1)
print(subdivided_mesh)
o3d.io.write_triangle_mesh(r'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer\reference_mesh/decimated_reference_mesh.obj', decimated_reference_mesh, write_vertex_normals=False, write_vertex_colors=False, write_triangle_uvs=False)
o3d.io.write_triangle_mesh(r'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer\reference_mesh/subdivided_reference_mesh.obj', subdivided_mesh, write_vertex_normals=False, write_vertex_colors=False, write_triangle_uvs=False)


20002
TriangleMesh with 5002 points and 10000 triangles.
TriangleMesh with 20002 points and 40000 triangles.


True

In [74]:
subdivided_mesh = o3d.geometry.TriangleMesh.subdivide_midpoint(decimated_reference_mesh, number_of_iterations=1)
o3d.io.write_triangle_mesh(r'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer\reference_mesh/subdivided_reference_mesh1.obj', subdivided_mesh, write_vertex_normals=False, write_vertex_colors=False, write_triangle_uvs=False)

True

In [91]:
reconstruct_dancer_5 = o3d.io.read_triangle_mesh(r'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\output\Dancer\reference/deformed_reference_mesh_005.obj')
original_dancer_5 = o3d.io.read_triangle_mesh(r'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer\meshes/dancer_fr0005.obj')
print(reconstruct_dancer_5)
print(original_dancer_5)
reconstruct_dancer_5.compute_vertex_normals()
original_dancer_5.compute_vertex_normals()
o3d.visualization.draw_geometries([reconstruct_dancer_5])

TriangleMesh with 4996 points and 10000 triangles.
TriangleMesh with 19724 points and 39380 triangles.


In [8]:
decimated_reconstructed_5 = o3d.geometry.TriangleMesh.simplify_quadric_decimation(reconstruct_dancer_5, 10000, boundary_weight= 8000)
fitting_mesh_dancer_5 = subdivide_surface_fitting(decimated_reconstructed_5, original_dancer_5, 1)
fitting_mesh_dancer_5.compute_vertex_normals()

o3d.visualization.draw_geometries([fitting_mesh_dancer_5])


TriangleMesh with 5002 points and 10000 triangles.
TriangleMesh with 20002 points and 40000 triangles.
TriangleMesh with 20002 points and 40000 triangles.


In [ ]:
o3d.io.write_triangle_mesh(r'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\output\Dancer\reference/fitting_mesh_dancer_5.obj', fitting_mesh_dancer_5, write_vertex_normals=False, write_vertex_colors=False, write_triangle_uvs=False)

In [5]:
GoF = 5
for i in range(0, GoF):
    offset = 5
    reconstruct_dancer_i = o3d.io.read_triangle_mesh(fr'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\output\Dancer\reference/deformed_reference_mesh_{i+offset:03}.obj')
    original_dancer_i = o3d.io.read_triangle_mesh(fr'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer\meshes/dancer_fr0{i+offset:03}.obj')
    #print(reconstruct_dancer_i)
    #print(original_dancer_i)
    reconstruct_dancer_i.compute_vertex_normals()
    original_dancer_i.compute_vertex_normals()
    #o3d.visualization.draw_geometries([reconstruct_dancer_i])
    decimated_reconstructed_i = o3d.geometry.TriangleMesh.simplify_quadric_decimation(reconstruct_dancer_i, 10000, boundary_weight= 8000)
    #print(decimated_reconstructed_i)
    o3d.io.write_triangle_mesh(fr'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\output\Dancer\reference/decimated_reconstructed_{i+offset:03}.obj', decimated_reconstructed_i, write_vertex_normals=False, write_vertex_colors=False, write_triangle_uvs=False)
    fitting_mesh_dancer_i = subdivide_surface_fitting(decimated_reconstructed_i, original_dancer_i, 1)
    #print(fitting_mesh_dancer_i)
    fitting_mesh_dancer_i.compute_vertex_normals()
    o3d.io.write_triangle_mesh(fr'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\output\Dancer\reference/fitting_mesh_dancer_{i+offset:03}.obj', fitting_mesh_dancer_i, write_vertex_normals=False, write_vertex_colors=False, write_triangle_uvs=False)
    #o3d.visualization.draw_geometries([fitting_mesh_dancer_i])

[Open3D WARNING] Write OBJ can not include triangle normals.
TriangleMesh with 19996 points and 40000 triangles.
[Open3D WARNING] Write OBJ can not include triangle normals.
[Open3D WARNING] Write OBJ can not include triangle normals.
TriangleMesh with 19996 points and 40000 triangles.
[Open3D WARNING] Write OBJ can not include triangle normals.
[Open3D WARNING] Write OBJ can not include triangle normals.
TriangleMesh with 19996 points and 40000 triangles.
[Open3D WARNING] Write OBJ can not include triangle normals.
[Open3D WARNING] Write OBJ can not include triangle normals.
TriangleMesh with 19996 points and 40000 triangles.
[Open3D WARNING] Write OBJ can not include triangle normals.
[Open3D WARNING] Write OBJ can not include triangle normals.
TriangleMesh with 19996 points and 40000 triangles.
[Open3D WARNING] Write OBJ can not include triangle normals.


In [4]:
def normalize_point_cloud(pcd, min_range=0, max_range=1):
    points = np.asarray(pcd.points)
    
    min_bound = points.min(axis=0)
    max_bound = points.max(axis=0)
    
    normalized_points = (points - min_bound) / (max_bound - min_bound)  
    normalized_points = normalized_points * (max_range - min_range) + min_range  
    
    pcd.points = o3d.utility.Vector3dVector(normalized_points)
    return pcd

def normalize_vertices(vertices, min_range=0, max_range=1):
    min_bound = np.min(vertices, axis=0)
    max_bound = np.max(vertices, axis=0)
    
    normalized_vertices = (vertices - min_bound) / (max_bound - min_bound)
    
    normalized_vertices = normalized_vertices * (max_range - min_range) + min_range
    
    return normalized_vertices

def compute_D1_psnr(original_mesh, decoded_mesh):
    
    original_vertices = np.array(original_mesh.vertices)
    #original_vertices = normalize_vertices(original_vertices)
    decoded_vertices = np.array(decoded_mesh.vertices)
    #decoded_vertices = normalize_vertices(decoded_vertices)
    
    pcd_original = o3d.geometry.PointCloud()
    pcd_original.points = o3d.utility.Vector3dVector(original_vertices)
    
    pcd_decoded = o3d.geometry.PointCloud()
    pcd_decoded.points = o3d.utility.Vector3dVector(decoded_vertices)
    pcd_tree = o3d.geometry.KDTreeFlann(pcd_decoded)
    

    #for i in range(0, len(decoded_vertices)):
    #    print('decoded', decoded_vertices[i])
    #for i in range(0, len(original_vertices)):
    #    print('original', original_vertices[i])
    MSE = 0
    for i in range(0, len(original_vertices)):
        [k, index, _] = pcd_tree.search_knn_vector_3d(original_vertices[i], 1)
        #print(original_vertices[i], decoded_vertices[index])
        MSE += np.square(original_vertices[i] - decoded_vertices[index])
        #if (np.square(original_vertices[i] - decoded_vertices[index]) != 0).any():
         #   print(original_vertices[i], decoded_vertices[index])
    MSE = np.mean(MSE)
    print(MSE)
    aabb = pcd_original.get_axis_aligned_bounding_box()
    min_bound = aabb.get_min_bound()

    max_bound = aabb.get_max_bound()

    signal_peak = np.linalg.norm(max_bound - min_bound)
    print(signal_peak)
    psnr = 20 * np.log10(signal_peak) - 10 * np.log10(MSE)
    
    return psnr

def read_triangle_mesh_with_trimesh(avatar_name,enable_post_processing=False):
    # EDIT: next 4 lines replace to maintain order even in case of degenerate and non referenced
    # scene_patch = trimesh.load(avatar_name,process=enable_post_processing)
    if enable_post_processing:
        scene_patch = trimesh.load(avatar_name,process=True)
    else:
        scene_patch = trimesh.load(avatar_name,process=False,maintain_order=True) 
    mesh = o3d.geometry.TriangleMesh(
        o3d.utility.Vector3dVector(scene_patch.vertices),
        o3d.utility.Vector3iVector(scene_patch.faces)
    ) 
    if scene_patch.vertex_normals.size:
        mesh.vertex_normals = o3d.utility.Vector3dVector(scene_patch.vertex_normals.copy())
    if scene_patch.visual.defined:
        # either texture or vertex colors if no uvs present.
        if scene_patch.visual.kind == 'vertex':
            mesh.vertex_colors = o3d.utility.Vector3dVector(scene_patch.visual.vertex_colors[:,:3]/255) # no alpha channel support
        elif scene_patch.visual.kind == 'texture':
            uv = scene_patch.visual.uv
            if uv.shape[0] == scene_patch.vertices.shape[0]:
                mesh.triangle_uvs = o3d.utility.Vector2dVector(uv[scene_patch.faces.flatten()])
            elif uv.shape[0] != scene_patch.faces.shape[0] * 3:
                assert False
            else:
                mesh.triangle_uvs = o3d.utility.Vector2dVector(uv)
                if scene_patch.visual.material is not None and scene_patch.visual.material.image is not None:
                    if scene_patch.visual.material.image.mode == 'RGB':
                        mesh.textures = [o3d.geometry.Image(np.asarray(scene_patch.visual.material.image))]
                    else:
                        assert False
        else:
            assert False
    return mesh


In [44]:
GoF = 5
for i in range(0, 5):
    offset = 5
    reconstruct_dancer_i = o3d.io.read_triangle_mesh(fr'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\output\Dancer\reference/deformed_reference_mesh_{i+offset:03}.obj', enable_post_processing=False)
    original_dancer_i = read_triangle_mesh_with_trimesh(fr'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer\meshes/dancer_fr0{i+offset:03}.obj')
    print(reconstruct_dancer_i)
    print(original_dancer_i)
    fitting_mesh_dancer_i = subdivide_surface_fitting(reconstruct_dancer_i, original_dancer_i, 1)
    fitting_mesh_dancer_i.compute_vertex_normals()
    o3d.io.write_triangle_mesh(fr'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\output\Dancer\reference/fitting_mesh_dancer_{i+offset:03}.obj', fitting_mesh_dancer_i, write_vertex_normals=False, write_vertex_colors=False, write_triangle_uvs=False)
    #o3d.visualization.draw_geometries([fitting_mesh_dancer_i])

TriangleMesh with 4996 points and 10000 triangles.
TriangleMesh with 20788 points and 39380 triangles.
TriangleMesh with 19996 points and 40000 triangles.
[Open3D WARNING] Write OBJ can not include triangle normals.
TriangleMesh with 4996 points and 10000 triangles.
TriangleMesh with 20833 points and 39411 triangles.
TriangleMesh with 19996 points and 40000 triangles.
[Open3D WARNING] Write OBJ can not include triangle normals.
TriangleMesh with 4996 points and 10000 triangles.
TriangleMesh with 20658 points and 39373 triangles.
TriangleMesh with 19996 points and 40000 triangles.
[Open3D WARNING] Write OBJ can not include triangle normals.
TriangleMesh with 4996 points and 10000 triangles.
TriangleMesh with 20778 points and 39392 triangles.
TriangleMesh with 19996 points and 40000 triangles.
[Open3D WARNING] Write OBJ can not include triangle normals.
TriangleMesh with 4996 points and 10000 triangles.
TriangleMesh with 20887 points and 39356 triangles.
TriangleMesh with 19996 points an

In [27]:
input_path = r'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer\reference_mesh/reference_mesh.obj'
input_path2 = r'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer\reference_mesh/order_reference_mesh.obj'
output_path = r'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer\reference_mesh/encode.drc'

## get error

In [20]:
def normalize_point_cloud(pcd, min_range=0, max_range=1):
    points = np.asarray(pcd.points)
    
    min_bound = points.min(axis=0)
    max_bound = points.max(axis=0)
    
    normalized_points = (points - min_bound) / (max_bound - min_bound)  
    normalized_points = normalized_points * (max_range - min_range) + min_range  
    
    pcd.points = o3d.utility.Vector3dVector(normalized_points)
    return pcd

def normalize_vertices(vertices, min_range=0, max_range=1):
    min_bound = np.min(vertices, axis=0)
    max_bound = np.max(vertices, axis=0)
    
    normalized_vertices = (vertices - min_bound) / (max_bound - min_bound)
    
    normalized_vertices = normalized_vertices * (max_range - min_range) + min_range
    
    return normalized_vertices

def compute_D1_psnr(original_mesh, decoded_mesh):
    
    original_vertices = np.array(original_mesh.vertices)
    #original_vertices = normalize_vertices(original_vertices)
    decoded_vertices = np.array(decoded_mesh.vertices)
    #decoded_vertices = normalize_vertices(decoded_vertices)
    
    pcd_original = o3d.geometry.PointCloud()
    pcd_original.points = o3d.utility.Vector3dVector(original_vertices)
    
    pcd_decoded = o3d.geometry.PointCloud()
    pcd_decoded.points = o3d.utility.Vector3dVector(decoded_vertices)
    pcd_tree = o3d.geometry.KDTreeFlann(pcd_decoded)
    

    
    MSE = 0
    for i in range(0, len(original_vertices)):
        [k, index, _] = pcd_tree.search_knn_vector_3d(original_vertices[i], 1)
        MSE += np.square(original_vertices[i] - decoded_vertices[index])
        #print(original_vertices[i], decoded_vertices[index])
    MSE = np.mean(MSE)
    #print(MSE)
    aabb = pcd_original.get_axis_aligned_bounding_box()
    min_bound = aabb.get_min_bound()

    max_bound = aabb.get_max_bound()

    signal_peak = np.linalg.norm(max_bound - min_bound)
    #print(signal_peak)
    psnr = 20 * np.log10(signal_peak) - 10 * np.log10(MSE)
    print(psnr)
    return psnr

In [46]:
def read_triangle_mesh_with_trimesh(avatar_name,enable_post_processing=False):
    # EDIT: next 4 lines replace to maintain order even in case of degenerate and non referenced
    # scene_patch = trimesh.load(avatar_name,process=enable_post_processing)
    if enable_post_processing:
        scene_patch = trimesh.load(avatar_name,process=True)
    else:
        scene_patch = trimesh.load(avatar_name,process=False,maintain_order=True) 
    mesh = o3d.geometry.TriangleMesh(
        o3d.utility.Vector3dVector(scene_patch.vertices),
        o3d.utility.Vector3iVector(scene_patch.faces)
    ) 
    if scene_patch.vertex_normals.size:
        mesh.vertex_normals = o3d.utility.Vector3dVector(scene_patch.vertex_normals.copy())
    if scene_patch.visual.defined:
        # either texture or vertex colors if no uvs present.
        if scene_patch.visual.kind == 'vertex':
            mesh.vertex_colors = o3d.utility.Vector3dVector(scene_patch.visual.vertex_colors[:,:3]/255) # no alpha channel support
        elif scene_patch.visual.kind == 'texture':
            uv = scene_patch.visual.uv
            if uv.shape[0] == scene_patch.vertices.shape[0]:
                mesh.triangle_uvs = o3d.utility.Vector2dVector(uv[scene_patch.faces.flatten()])
            elif uv.shape[0] != scene_patch.faces.shape[0] * 3:
                assert False
            else:
                mesh.triangle_uvs = o3d.utility.Vector2dVector(uv)
                if scene_patch.visual.material is not None and scene_patch.visual.material.image is not None:
                    if scene_patch.visual.material.image.mode == 'RGB':
                        mesh.textures = [o3d.geometry.Image(np.asarray(scene_patch.visual.material.image))]
                    else:
                        assert False
        else:
            assert False
    return mesh


In [47]:
reconstruct = read_triangle_mesh_with_trimesh(fr'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\output\Dancer\reference/fitting_mesh_dancer_005.obj')
print(reconstruct)
original_dancer = read_triangle_mesh_with_trimesh(fr'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer\meshes/dancer_fr0005.obj')
print(original_dancer)
#d1 = compute_D1_psnr(original_dancer, reconstruct)
#print(d1)

TriangleMesh with 19996 points and 40000 triangles.
TriangleMesh with 20788 points and 39380 triangles.


In [48]:
d1 = compute_D1_psnr(reconstruct, original_dancer)
print(reconstruct, original_dancer)
print(d1)

0.0
2.153985834371712
TriangleMesh with 19996 points and 40000 triangles. TriangleMesh with 20788 points and 39380 triangles.
inf


C:\Users\Windows\AppData\Local\Temp\ipykernel_31836\289853084.py:53: RuntimeWarning: divide by zero encountered in log10
  psnr = 20 * np.log10(signal_peak) - 10 * np.log10(MSE)


In [49]:
reference_mesh = read_triangle_mesh_with_trimesh(r'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer/reference_mesh.obj')
print(reference_mesh)
decimated_reference_mesh = o3d.geometry.TriangleMesh.simplify_quadric_decimation(reference_mesh, 10000, boundary_weight= 8000)
print(decimated_reference_mesh)
decimated_reference_mesh_vertices = np.array(decimated_reference_mesh.vertices)
o3d.io.write_triangle_mesh(r'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer\reference_mesh/decimated_reference_mesh.obj', decimated_reference_mesh, write_vertex_normals=False, write_vertex_colors=False, write_triangle_uvs=False)
loaded_decimated_reference_mesh = read_triangle_mesh_with_trimesh(r'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer\reference_mesh/decimated_reference_mesh.obj', enable_post_processing=False)
print(loaded_decimated_reference_mesh)
loaded_decimated_reference_mesh_vertices = np.array(loaded_decimated_reference_mesh.vertices)
loaded_decimated_reference_mesh_vertices

TriangleMesh with 20002 points and 40000 triangles.
TriangleMesh with 5002 points and 10000 triangles.
TriangleMesh with 4996 points and 10000 triangles.


array([[ 0.0758296,  0.911741 , -0.472649 ],
       [ 0.0902357,  0.911741 , -0.48194  ],
       [ 0.0901355,  0.903519 , -0.477867 ],
       ...,
       [ 0.0342721,  0.614468 ,  0.0390702],
       [ 0.289648 , -0.456912 ,  0.0853124],
       [-0.201152 ,  0.548629 ,  0.0756965]])

In [50]:
reference_mesh = read_triangle_mesh_with_trimesh(r'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer/reference_mesh.obj')
print(reference_mesh)
decimated_reference_mesh = o3d.geometry.TriangleMesh.simplify_quadric_decimation(reference_mesh, 10000, boundary_weight= 8000)
print(decimated_reference_mesh)
decimated_reference_mesh_vertices = np.array(decimated_reference_mesh.vertices)


TriangleMesh with 20002 points and 40000 triangles.
TriangleMesh with 5002 points and 10000 triangles.


In [19]:
decimated_reference_mesh_vertices

array([[ 0.07582965,  0.911741  , -0.472649  ],
       [ 0.09023573,  0.911741  , -0.48194025],
       [ 0.09013545,  0.9035185 , -0.477867  ],
       ...,
       [ 0.0342721 ,  0.614468  ,  0.0390702 ],
       [ 0.2896475 , -0.456912  ,  0.08531235],
       [-0.201152  ,  0.548629  ,  0.0756965 ]])

In [51]:
subdivided_decimated_reference_mesh = o3d.geometry.TriangleMesh.subdivide_midpoint(decimated_reference_mesh, number_of_iterations=1)
print(subdivided_decimated_reference_mesh)
subdivided_decimated_reference_mesh_vertices = np.array(subdivided_decimated_reference_mesh.vertices)

TriangleMesh with 20002 points and 40000 triangles.


In [21]:
subdivided_decimated_reference_mesh_vertices

array([[ 0.07582965,  0.911741  , -0.472649  ],
       [ 0.09023573,  0.911741  , -0.48194025],
       [ 0.09013545,  0.9035185 , -0.477867  ],
       ...,
       [ 0.18076125, -0.41358662,  0.30659475],
       [-0.04329998,  0.79348337,  0.31874463],
       [-0.05850105,  0.398496  ,  0.316869  ]])

## get displacements

In [52]:
decimated_reference_mesh = loaded_decimated_reference_mesh
print(decimated_reference_mesh)

TriangleMesh with 4996 points and 10000 triangles.


In [13]:
dataset = 'Dancer'
GoF = 5

In [54]:
for i in range(0, GoF):
    offset = 5
    reconstruct_dancer_i = read_triangle_mesh_with_trimesh(fr'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\output\Dancer\reference/deformed_reference_mesh_{i+offset:03}.obj')
    original_dancer_i = read_triangle_mesh_with_trimesh(fr'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer\meshes/dancer_fr0{i+offset:03}.obj')

    fitting_mesh_dancer_i = subdivide_surface_fitting(reconstruct_dancer_i, original_dancer_i, 1)
    print(fitting_mesh_dancer_i)
    o3d.io.write_triangle_mesh(fr'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\output\Dancer\reference/fitting_mesh_dancer_{i+offset:03}.obj', fitting_mesh_dancer_i, write_vertex_normals=False, write_vertex_colors=False, write_triangle_uvs=False)
    #o3d.visualization.draw_geometries([fitting_mesh_dancer_i])

TriangleMesh with 19996 points and 40000 triangles.
TriangleMesh with 19996 points and 40000 triangles.
[Open3D WARNING] Write OBJ can not include triangle normals.
TriangleMesh with 19996 points and 40000 triangles.
TriangleMesh with 19996 points and 40000 triangles.
[Open3D WARNING] Write OBJ can not include triangle normals.
TriangleMesh with 19996 points and 40000 triangles.
TriangleMesh with 19996 points and 40000 triangles.
[Open3D WARNING] Write OBJ can not include triangle normals.
TriangleMesh with 19996 points and 40000 triangles.
TriangleMesh with 19996 points and 40000 triangles.
[Open3D WARNING] Write OBJ can not include triangle normals.
TriangleMesh with 19996 points and 40000 triangles.
TriangleMesh with 19996 points and 40000 triangles.
[Open3D WARNING] Write OBJ can not include triangle normals.


In [56]:
subdivided_decimated_reference_mesh = o3d.geometry.TriangleMesh.subdivide_midpoint(decimated_reference_mesh, number_of_iterations=1)
print(subdivided_decimated_reference_mesh)
subdivided_decimated_reference_mesh_vertices = np.array(subdivided_decimated_reference_mesh.vertices)
subdivided_decimated_reference_mesh_vertices

TriangleMesh with 19996 points and 40000 triangles.


array([[ 0.0758296 ,  0.911741  , -0.472649  ],
       [ 0.0902357 ,  0.911741  , -0.48194   ],
       [ 0.0901355 ,  0.903519  , -0.477867  ],
       ...,
       [ 0.1807615 , -0.4135865 ,  0.306595  ],
       [-0.0433    ,  0.7934835 ,  0.3187445 ],
       [-0.05850105,  0.398496  ,  0.316869  ]])

In [57]:
displacements = []
for i in range(0, GoF):
    offset = 5
    fitting_mesh_dancer_i = read_triangle_mesh_with_trimesh(fr'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\output\Dancer\reference/fitting_mesh_dancer_{i+offset:03}.obj')
    print(fitting_mesh_dancer_i, subdivided_decimated_reference_mesh)
    fitting_mesh_vertices = np.array(fitting_mesh_dancer_i.vertices)
    displacement_i = subdivided_decimated_reference_mesh_vertices - fitting_mesh_vertices
    
    np.savetxt(fr'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\output\Dancer\reference/displacements_{dataset}_{i+offset:03}.txt', displacement_i, fmt='%8f')
    displacements.append(displacement_i)

TriangleMesh with 19996 points and 40000 triangles. TriangleMesh with 19996 points and 40000 triangles.
TriangleMesh with 19996 points and 40000 triangles. TriangleMesh with 19996 points and 40000 triangles.
TriangleMesh with 19996 points and 40000 triangles. TriangleMesh with 19996 points and 40000 triangles.
TriangleMesh with 19996 points and 40000 triangles. TriangleMesh with 19996 points and 40000 triangles.
TriangleMesh with 19996 points and 40000 triangles. TriangleMesh with 19996 points and 40000 triangles.


In [58]:
input_reference_mesh_path = r'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer\reference_mesh/decimated_reference_mesh.obj'
input_decimated_reference_mesh = read_triangle_mesh_with_trimesh(input_reference_mesh_path, enable_post_processing=False)
np.array(input_decimated_reference_mesh.vertices)

array([[ 0.0758296,  0.911741 , -0.472649 ],
       [ 0.0902357,  0.911741 , -0.48194  ],
       [ 0.0901355,  0.903519 , -0.477867 ],
       ...,
       [ 0.0342721,  0.614468 ,  0.0390702],
       [ 0.289648 , -0.456912 ,  0.0853124],
       [-0.201152 ,  0.548629 ,  0.0756965]])

In [59]:
output_path = r'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer\reference_mesh/1.drc'
result = subprocess.run([
                            r'G:\Github\draco\build\Debug\draco_encoder',
                            '-i', input_reference_mesh_path,
                            '-o', output_path,
                            '-qp', str('14'),
                            '-cl', '7'
                            ], capture_output=True, text=True)
print(result.stdout)

Encoder options:
  Compression level = 7
  Positions: Quantization = 14 bits

Encoded mesh saved to G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer\reference_mesh/1.drc (47 ms to encode).

Encoded size = 18111 bytes

For better compression, increase the compression level up to '-cl 10' .




In [60]:
result = subprocess.run([
                            r'G:\Github\draco\build\Debug\draco_decoder',
                            '-i', output_path,
                            '-o', r'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer\reference_mesh/decode.obj'
                            ], capture_output=True, text=True)
print(result.stdout)

Decoded geometry saved to G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer\reference_mesh/decode.obj (13 ms to decode)



In [61]:
decoded_mesh = read_triangle_mesh_with_trimesh(r'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer\reference_mesh/decode.obj', enable_post_processing=False)
print(decoded_mesh)
np.array(decoded_mesh.vertices)

TriangleMesh with 4996 points and 10000 triangles.


array([[-0.053439,  1.16511 ,  0.158964],
       [-0.045483,  1.176458,  0.163878],
       [-0.046887,  1.157154,  0.161655],
       ...,
       [-0.423731,  0.700051, -0.446256],
       [-0.430517,  0.700752, -0.440874],
       [-0.43859 ,  0.686947, -0.438768]])

In [62]:
test = o3d.geometry.TriangleMesh.subdivide_midpoint(decoded_mesh, number_of_iterations=1)
print(test)
subdivided_mesh = o3d.geometry.TriangleMesh.subdivide_midpoint(input_decimated_reference_mesh, number_of_iterations=1)
print(subdivided_mesh)
test_vertices = np.array(test.vertices)
subdivided_vertices = np.array(subdivided_mesh.vertices)
test.compute_vertex_normals()
subdivided_mesh.compute_vertex_normals()
test.paint_uniform_color([1,0,0])
o3d.visualization.draw_geometries([test, subdivided_mesh])


TriangleMesh with 19996 points and 40000 triangles.
TriangleMesh with 19996 points and 40000 triangles.


In [69]:
displacement = np.loadtxt(fr'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\output\Dancer\reference/displacements_{dataset}_005.txt')

In [2]:
#displacement = np.loadtxt(fr'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\output\Dancer\reference/displacements_{dataset}_005.txt')
dis_mesh = o3d.io.read_point_cloud(fr'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer\reference_mesh/decoded_displacements.ply')
displacement = np.array(dis_mesh.points)
displacement

array([[ 0.01549193,  0.04196607,  0.00506445],
       [ 0.18973413,  0.03460371,  0.00408281],
       [ 0.01352865,  0.03460371,  0.00997268],
       ...,
       [-0.01052171, -0.08564795, -0.07690302],
       [-0.01935652, -0.09202866, -0.093591  ],
       [-0.01788405, -0.08761124, -0.08573782]])

In [70]:
pcd_indexer = o3d.geometry.PointCloud()
pcd_indexer.points = o3d.utility.Vector3dVector(subdivided_mesh.vertices)
pcd_tree = o3d.geometry.KDTreeFlann(pcd_indexer)
re_test_vertices = deepcopy(test_vertices)
for i in range(0, len(test_vertices)):
    [k, index, _] = pcd_tree.search_knn_vector_3d(test_vertices[i], 1)
    re_test_vertices[i] -= displacement[index[0]]
    #print(test_vertices[i], subdivided_vertices[index], index[0])
    #print(displacement[index[0]])
test.vertices = o3d.utility.Vector3dVector(re_test_vertices)
re_test_vertices

array([[-0.08141  ,  1.15301  ,  0.164006 ],
       [-0.073311 ,  1.166138 ,  0.168474 ],
       [-0.07537  ,  1.143984 ,  0.166482 ],
       ...,
       [-0.111114 ,  0.8755505, -0.4735835],
       [-0.121165 ,  0.878556 , -0.4781065],
       [-0.1246895,  0.8805475, -0.482688 ]])

In [71]:
test.paint_uniform_color([0.7,0.7,0.7])
original_dancer.paint_uniform_color([1,0,0])
o3d.visualization.draw_geometries([test])
print(test)

TriangleMesh with 19996 points and 40000 triangles.


In [66]:
d1 = max(compute_D1_psnr(original_dancer, test), compute_D1_psnr(test, original_dancer))
print(d1)

0.10430502592332601
2.1541824332553636
1.4090589027275816e-05
2.154070430518754
55.17590649937894


In [111]:
print(compute_D1_psnr(original_dancer, test))

0.10430502592332601
2.1541824332553636
16.482597246913755


In [108]:
import re
import os

original_input_dir = os.path.join(r'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data', os.path.join(dataset, 'meshes'))
decoded_input_dir = os.path.join(r'G:\PycharmProjects\Mesh_Editing\Results\decode_Draco', dataset)

files = os.listdir(original_input_dir)
original_obj_files = [file for file in files if file.endswith('.obj')]
def extract_number(filename):
    match = re.search(r'(\d+)', filename)
    return int(match.group(1)) if match else float('inf')

original_obj_files.sort(key=extract_number)
    
qps = range(6,15)
result_file_path = os.path.join(r'G:\PycharmProjects\Mesh_Editing\Results', f'{dataset}_D1_PSNR_results.txt')
with open(result_file_path, 'w') as result_file:
    for qp in qps:
        d1_psnr_values = []
        decoded_obj_path = os.path.join(decoded_input_dir, f'{dataset}_qp_{qp}')
        files = os.listdir(decoded_obj_path)
        decoded_obj_files = [file for file in files if file.endswith('.obj')]
        for i in range(len(original_obj_files)):
            original_mesh = o3d.io.read_triangle_mesh(os.path.join(original_input_dir, original_obj_files[i]))
            decoded_mesh = o3d.io.read_triangle_mesh(os.path.join(decoded_obj_path, decoded_obj_files[i]))
            d1_psnr = max(compute_D1_psnr(original_mesh, decoded_mesh), compute_D1_psnr(decoded_mesh, original_mesh))
            d1_psnr_values.append(d1_psnr)
            
        avg_psnr_5 = np.mean(d1_psnr_values[:5])
        avg_psnr_10 = np.mean(d1_psnr_values[:10])
        avg_psnr_15 = np.mean(d1_psnr_values[:15])
        avg_psnr = np.mean(d1_psnr_values)

        result_file.write(f"qp={qp}, Number of frames: {5}, Average D1 PSNR: {avg_psnr_5}\n")
        result_file.write(f"qp={qp}, Number of frames: {10}, Average D1 PSNR: {avg_psnr_10}\n")
        result_file.write(f"qp={qp}, Number of frames: {15}, Average D1 PSNR: {avg_psnr_15}\n")
        
        print(f"qp={qp}, Number of frames: {len(original_obj_files)}, Average D1 PSNR: {avg_psnr}\n")

1.373884003623045
2.154182467483461
0.14703490331987157
2.1506494733131905
1.3415661943519048
2.1489807178796845
0.14471352531646292
2.1509806482521374
1.3400588257808106
2.1495747096785562
0.14400835193956504
2.144647842629626
1.3438243599327462
2.160141938959326
0.13957657459180384
2.156660888360939
1.3662830963046815
2.1679880499474744
0.14038101639807626
2.168679591817534
1.3704191574927174
2.1751943388149377
0.14366281972624328
2.1824967399118087
1.4091069366071851
2.1800171671271986
0.14483653368265623
2.180429837686529
1.4161677046357497
2.180608537220912
0.154992412286531
2.1858150078686784
1.4055336725229919
2.171643552469872
0.14492850563013007
2.1798360441355893
1.3721963475788541
2.159199371971853
0.14938904788201993
2.1559997437460754
1.3480193958179514
2.1595790473832817
0.14219138552671085
2.1611988211301267
1.3346837051075386
2.158615660372041
0.14522228482064148
2.1580751169147705
1.349168662067357
2.1658708183857107
0.14536323792466413
2.1641225798591606
1.36646181836

KeyboardInterrupt: 

In [5]:
pcd = o3d.geometry.PointCloud()
points = displacement
pcd.points = o3d.utility.Vector3dVector(points)
print(pcd)
o3d.io.write_point_cloud(r'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer\reference_mesh/dis.ply', pcd)

PointCloud with 19996 points.


True

In [124]:
points=np.asarray(pcd.points)
dtype = o3d.core.float32
p_tensor = o3d.core.Tensor(points, dtype=dtype)
pc = o3d.t.geometry.PointCloud(p_tensor)
o3d.t.io.write_point_cloud(r'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer\reference_mesh/dis_32.ply', pc, write_ascii=True)

True

In [7]:
testpcd = o3d.io.read_point_cloud(r'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer\reference_mesh/dis.ply')
print(testpcd)
np.array(testpcd.points)[0]

PointCloud with 19996 points.


array([0.01549193, 0.04196607, 0.00506445])

In [81]:
qp = 8
result = subprocess.run([
                            r'G:\Github\draco\build\Debug\draco_encoder',
                            #r'G:\Github\draco\buildforSequenceEncoding\Debug\draco_encoder',
                            '-point_cloud',
                            '-i', r'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer\reference_mesh/dis_32.ply',
                            '-o', r'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer\reference_mesh/dis.drc',
                            '-qp', str(qp),
                            '-cl', '10'
                            ], capture_output=True, text=True)
print(result.stdout)

result = subprocess.run([
                            r'G:\Github\draco\build\Debug\draco_encoder',
                            #r'G:\Github\draco\buildforSequenceEncoding\Debug\draco_encoder',
                            '-point_cloud',
                            '-i', r'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer\reference_mesh/dis_32.ply',
                            '-o', r'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer\reference_mesh/dis_0.drc',
                            '-qp', str(qp),
                            '-cl', '0'
                            ], capture_output=True, text=True)
print(result.stdout)

Encoder options:
  Compression level = 10
  Positions: Quantization = 8 bits

Encoded point cloud saved to G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer\reference_mesh/dis.drc (23 ms to encode).

Encoded size = 7679 bytes


Encoder options:
  Compression level = 0
  Positions: Quantization = 8 bits

Encoded point cloud saved to G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer\reference_mesh/dis_0.drc (5 ms to encode).

Encoded size = 33516 bytes

For better compression, increase the compression level up to '-cl 10' .




In [82]:
result = subprocess.run([
                            r'G:\Github\draco\build\Debug\draco_decoder',
                            '-i', r'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer\reference_mesh/dis_0.drc',
                            '-o', r'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer\reference_mesh/decoded_displacements.ply'
                            ], capture_output=True, text=True)
print(result.stdout)

Decoded geometry saved to G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer\reference_mesh/decoded_displacements.ply (3 ms to decode)



In [9]:
def calculate_bitrate(file_size, duration):
    return file_size * 8 / duration

In [10]:
number_frames = 5
frame_rate = 30

In [83]:
total_size = 0
displacement_file_path = r'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer\reference_mesh/dis.drc'
displacement_file_size = os.path.getsize(displacement_file_path)
total_size += displacement_file_size * number_frames
reference_mesh_file_path = r'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer\reference_mesh/1.drc'
reference_mesh_file_size = os.path.getsize(reference_mesh_file_path)
total_size += reference_mesh_file_size
total_duration = number_frames / frame_rate
overall_bitrate = calculate_bitrate(total_size, total_duration)

print(f"Total Size of {number_frames} DRC Files: {total_size} bytes")
print(f"Overall Bitrate: {overall_bitrate} bits per second")

bitrate_kbps = overall_bitrate / 1000
bitrate_mbps = overall_bitrate / 1000000


print(f"Overall Bitrate: {bitrate_kbps:.2f} Kbps")
print(f"Overall Bitrate: {bitrate_mbps:.2f} Mbps")

Total Size of 5 DRC Files: 56506 bytes
Overall Bitrate: 2712288.0 bits per second
Overall Bitrate: 2712.29 Kbps
Overall Bitrate: 2.71 Mbps


In [39]:
decoded_mesh = read_triangle_mesh_with_trimesh(r'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer\reference_mesh/decode.obj', enable_post_processing=False)
print(decoded_mesh)
np.array(decoded_mesh.vertices)
subdivided_decoded_mesh = o3d.geometry.TriangleMesh.subdivide_midpoint(decoded_mesh, number_of_iterations=1)
print(subdivided_decoded_mesh)
input_decimated_reference_mesh = read_triangle_mesh_with_trimesh(r'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer\reference_mesh/decimated_reference_mesh.obj', enable_post_processing=False)
subdivided_mesh = o3d.geometry.TriangleMesh.subdivide_midpoint(input_decimated_reference_mesh, number_of_iterations=1)
print(subdivided_mesh)
original_dancer = read_triangle_mesh_with_trimesh(fr'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer\meshes/dancer_fr0005.obj')
print(original_dancer)

TriangleMesh with 4996 points and 10000 triangles.
TriangleMesh with 19996 points and 40000 triangles.
TriangleMesh with 19996 points and 40000 triangles.
TriangleMesh with 20788 points and 39380 triangles.


In [84]:
original_displacements = np.loadtxt(fr'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\output\Dancer\reference/displacements_{dataset}_005.txt')
decoded_displacements = o3d.io.read_point_cloud(fr'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer\reference_mesh/decoded_displacements.ply')
dis_ply = o3d.io.read_point_cloud(fr'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer\reference_mesh/decoded_displacements.ply')
displacement = np.array(dis_ply.points)
displacement

array([[ 0.20739794,  0.0246602 ,  0.00486234],
       [ 0.20542887,  0.0246602 ,  0.0068314 ],
       [ 0.20542887,  0.0246602 ,  0.00880047],
       ...,
       [ 0.0203366 ,  0.00300047,  0.00486234],
       [-0.0032922 , -0.00881393,  0.00486234],
       [ 0.00458407,  0.0010314 , -0.00301393]])

In [85]:
decoded_mesh = read_triangle_mesh_with_trimesh(r'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer\reference_mesh/decode.obj', enable_post_processing=False)
print(decoded_mesh)
np.array(decoded_mesh.vertices)
subdivided_decoded_mesh = o3d.geometry.TriangleMesh.subdivide_midpoint(decoded_mesh, number_of_iterations=1)
print(subdivided_decoded_mesh)
input_decimated_reference_mesh = read_triangle_mesh_with_trimesh(r'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer\reference_mesh/decimated_reference_mesh.obj', enable_post_processing=False)
subdivided_mesh = o3d.geometry.TriangleMesh.subdivide_midpoint(input_decimated_reference_mesh, number_of_iterations=1)
print(subdivided_mesh)
original_dancer = read_triangle_mesh_with_trimesh(fr'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer\meshes/dancer_fr0005.obj')
print(original_dancer)
decoded_mesh_vertices = np.array(decoded_mesh.vertices)
subdivided_decoded_mesh_vertices = np.array(subdivided_decoded_mesh.vertices)


displacement = np.array(decoded_displacements.points)

dis_indexer = o3d.geometry.PointCloud()
dis_indexer.points = o3d.utility.Vector3dVector(original_displacements)
dis_tree = o3d.geometry.KDTreeFlann(dis_indexer)

pcd_indexer = o3d.geometry.PointCloud()
pcd_indexer.points = o3d.utility.Vector3dVector(subdivided_mesh.vertices)
pcd_tree = o3d.geometry.KDTreeFlann(pcd_indexer)

reordered_vertices = deepcopy(subdivided_decoded_mesh_vertices)

for i in range(0, len(subdivided_decoded_mesh_vertices)):
    [k, index, _] = pcd_tree.search_knn_vector_3d(subdivided_decoded_mesh_vertices[i], 1)
    [j, dis_index, _] = dis_tree.search_knn_vector_3d(original_displacements[index[0]], 1)
    #print(displacement[dis_index], original_displacements[index[0]])
    reordered_vertices[i] -= displacement[dis_index[0]]
    

subdivided_decoded_mesh.vertices = o3d.utility.Vector3dVector(reordered_vertices)
subdivided_decoded_mesh.compute_vertex_normals()
o3d.visualization.draw_geometries([subdivided_decoded_mesh])
d1 = max(compute_D1_psnr(original_dancer, subdivided_decoded_mesh), compute_D1_psnr(subdivided_decoded_mesh, original_dancer))
print(d1)

TriangleMesh with 4996 points and 10000 triangles.
TriangleMesh with 19996 points and 40000 triangles.
TriangleMesh with 19996 points and 40000 triangles.
TriangleMesh with 20788 points and 39380 triangles.
16.42492108690567
28.556041826916335
28.556041826916335
